In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from math import pi, log, exp

Importação dos resultados do ensaio

In [5]:
df = pd.read_csv('D:/Usuario/Documents/MeusDados/modulo_resiliencia/modelo.csv', sep=';')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Usuario/Documents/MeusDados/modulo_resiliencia/modelo.csv'

In [ ]:
df

Calculo dos resultados

In [ ]:
e_list = list()
sd_list = list()
mr_list = list()
oct_list = list()
for i in df.index:
    e = (float(df['LVDT-A (mm)'][i])+float(df['LVDT-B (mm)'][i]))/2/200
    sd =  (float(df['Carga (kgf)'][i])*9.81)/(100**2*pi/4)
    mr = sd/e
    oct = (sd+3*float(df['s3 (MPa)'][i]))
    e_list.append(e)
    sd_list.append(sd)
    mr_list.append(mr)
    oct_list.append(oct)
df['e'] = e_list
df['sd (MPa)'] = sd_list
df['oct (MPa)'] = oct_list
df['MR (MPa)'] = mr_list


In [ ]:
df_result = df.copy()

Preparação para regressão potêncial

In [ ]:
#transformação dos parâmetros de entrada para a base ln
ln_mr_list = list()
ln_e_list = list()
ln_s3_list = list()
ln_sd_list = list()
ln_oct_list = list()
for i in df.index:
    ln_mr = log(float(df['MR (MPa)'][i]))
    ln_e = log(float(df['e'][i]))
    ln_s3 = log(float(df['s3 (MPa)'][i]))
    ln_sd = log(float(df['sd (MPa)'][i]))
    ln_oct = log(float(df['oct (MPa)'][i]))
    ln_mr_list.append(ln_mr)
    ln_e_list.append(ln_e)
    ln_s3_list.append(ln_s3)
    ln_sd_list.append(ln_sd)
    ln_oct_list.append(ln_oct)
df['ln_mr'] = ln_mr_list
df['ln_e'] = ln_e_list
df['ln_s3'] = ln_s3_list
df['ln_sd'] = ln_sd_list
df['ln_oct'] = ln_oct_list

In [ ]:
#regressão linear dos parâmetro na base ln
y_e = df.iloc[:, 9].values
y_mr = df.iloc[:, 8].values

In [ ]:
x_s3 = df.iloc[:, 10].values
x_s3 = x_s3.reshape(-1,1)
x_sd = df.iloc[:, 11].values
x_sd = x_sd.reshape(-1,1)
x_oct = df.iloc[:, 12].values
x_oct = x_oct.reshape(-1,1)
x_comp = df.iloc[:, 10:12].values

In [ ]:
from sklearn.linear_model import LinearRegression
regressor_s3 = LinearRegression()
regressor_sd = LinearRegression()
regressor_oct = LinearRegression()
regressor_comp = LinearRegression()
regressor_s3.fit(x_s3, y_mr)
regressor_sd.fit(x_sd, y_e)
regressor_oct.fit(x_oct, y_mr)
regressor_comp.fit(x_comp, y_e)

Regressão com k3 e k4 iguais a zero

In [ ]:
k1_s3 = regressor_s3.intercept_
k2 = regressor_s3.coef_[0]
print(f'Interseção:{k1_s3}, Variável X1:{k2}')

Regressão com k2 e k4 iguais a zero

In [ ]:
k1_sd = regressor_sd.intercept_
k3 = regressor_sd.coef_[0]
print(f'Interseção:{k1_sd}, Variável X1:{k3}')

Regressão com k2 e k3 iguais a zero

In [ ]:
k1_oct = regressor_oct.intercept_
k4 = regressor_oct.coef_[0]
print(f'Interseção:{k1_oct}, Variável X1:{k4}')

Regressão do modelo composto com k4 igual a zero

In [ ]:
k1_comp = regressor_comp.intercept_
k2_comp = regressor_comp.coef_[0]
k3_comp = regressor_comp.coef_[1]
print(f'Interseção:{k1_comp}, Variável X1:{k2_comp}, Variável X2:{k3_comp}')

Tranformação para parâmetros potênciais

In [ ]:
#modelo dependente da tensão de confinamento
K1_s3 = float(exp(k1_s3))
K2 = k2
eq_s3 = (f'MR = {K1_s3:.0f}*s3^{K2:.4f}')
print(eq_s3)
score_s3 = regressor_s3.score(x_s3, y_mr)
print(f'R² = {score_s3:.2}')

In [ ]:
#modelo dependente da tensão desvio
K1_sd = 1/exp(k1_sd)
K3 = 1-k3
eq_sd = (f'MR = {K1_sd:.0f}*sd^{K3:.4f}')
print(eq_sd)
score_sd = regressor_sd.score(x_sd, y_e)
print(f'R² = {score_sd:.2}')

In [ ]:
#modelo dependente do invariante de tensões
K1_oct = exp(k1_oct)
K4 = k4
eq_oct = (f'MR = {K1_oct:.0f}*oct^{K4:.4f}')
print(eq_oct)
score_oct = regressor_oct.score(x_oct, y_mr)
print(f'R² = {score_oct:.2}')

In [ ]:
#modelo composto
K1_comp = 1/exp(k1_comp)
K2_comp = -k2_comp
K3_comp = 1-k3_comp
eq_comp = (f'MR = {K1_comp:.0f}*s3^{K2_comp:.4f}*sd^{K3_comp:.4f}')
print(eq_comp)
score_comp = regressor_comp.score(x_comp, y_e)
print(f'R² = {score_comp:.2}')

Previsões de MR

In [ ]:
mr_s3_list = list()
mr_sd_list = list()
mr_oct_list = list()
mr_comp_list = list()
for i in df.index:
    mr_s3_cal = K1_s3*(float(df['s3 (MPa)'][i]))**K2
    mr_sd_cal = K1_sd*(float(df['sd (MPa)'][i]))**K3
    mr_oct_cal = K1_oct*(float(df['oct (MPa)'][i]))**K4
    mr_comp_cal = K1_comp*((float(df['s3 (MPa)'][i]))**K2_comp)*((float(df['sd (MPa)'][i]))**K3_comp)
    #print(mr_s3_cal, mr_sd_cal, mr_oct_cal, mr_comp_cal)
    mr_s3_list.append(mr_s3_cal)
    mr_sd_list.append(mr_sd_cal)
    mr_oct_list.append(mr_oct_cal)
    mr_comp_list.append(mr_comp_cal)       
df['mr_s3_cal'] = mr_s3_list
df['mr_sd_cal'] = mr_sd_list
df['mr_oct_cal'] = mr_oct_list
df['mr_comp_cal'] = mr_comp_list

Gráficos

In [ ]:
fig_s3 = plt.figure()
x = df.filter(items=['s3 (MPa)'])
y = df.filter(items=['MR (MPa)'])
y_pre = df.filter(items=['mr_s3_cal'])
plt.scatter(x, y, vmin=100)
plt.plot(x, y_pre, color='red')
plt.title('Módulo de Resiliência')
plt.xlabel('Tensão de confinamento')
plt.ylabel('Módulo de resiliência')
plt.xlim(0,0.15)
plt.ylim(0,500)

In [ ]:
fig_sd = plt.figure()
x = df.filter(items=['sd (MPa)'])
y = df.filter(items=['MR (MPa)'])
y_pre = df.filter(items=['mr_sd_cal'])
plt.scatter(x, y, vmin=100)
plt.plot(x, y_pre, color='red')
plt.title('Módulo de Resiliência')
plt.xlabel('Tensão de desvio')
plt.ylabel('Módulo de resiliência')
plt.xlim(0, 0.5)
plt.ylim(0, 500)

In [ ]:
fig_oct = plt.figure()
x = df.filter(items=['oct (MPa)'])
y = df.filter(items=['MR (MPa)'])
y_pre = df.filter(items=['mr_oct_cal'])
plt.scatter(x, y, vmin=100)
plt.plot(x, y_pre, color='red')
plt.title('Módulo de Resiliência')
plt.xlabel('Invariante de tensão')
plt.ylabel('Módulo de resiliência')
plt.xlim(0, 0.5)
plt.ylim(0, 500)

Métricas de avaliação dos modelos

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
mae_s3 = mean_absolute_error((df.iloc[:, 7].values), (df.iloc[:, 13].values))
msr_s3 = mean_squared_error((df.iloc[:, 7].values), (df.iloc[:, 13].values))
r2_s3 = r2_score((df.iloc[:, 7].values), (df.iloc[:, 13].values))
print(f'O modelo dependente de s3 tem o R² de {r2_s3:.2f}, o MAE de {mae_s3:.2f} e o MSR de {msr_s3:.2f}.')

In [ ]:
mae_sd = mean_absolute_error((df.iloc[:, 7].values), (df.iloc[:, 14].values))
msr_sd = mean_squared_error((df.iloc[:, 7].values), (df.iloc[:, 14].values))
r2_sd = r2_score((df.iloc[:, 7].values), (df.iloc[:, 14].values))
print(f'O modelo dependente de sd tem o R² de {r2_sd:.2f}, o MAE de {mae_sd:.2f} e o MSR de {msr_sd:.2f}.')

In [ ]:
mae_oct = mean_absolute_error((df.iloc[:, 7].values), (df.iloc[:, 15].values))
msr_oct = mean_squared_error((df.iloc[:, 7].values), (df.iloc[:, 15].values))
r2_oct = r2_score((df.iloc[:, 7].values), (df.iloc[:, 15].values))
print(f'O modelo dependente de oct tem o R² de {r2_oct:.2f}, o MAE de {mae_oct:.2f} e o MSR de {msr_oct:.2f}.')

In [ ]:
mae_comp = mean_absolute_error((df.iloc[:, 7].values), (df.iloc[:, 16].values))
msr_comp = mean_squared_error((df.iloc[:, 7].values), (df.iloc[:, 16].values))
r2_comp = r2_score((df.iloc[:, 7].values), (df.iloc[:, 16].values))
print(f'O modelo composto tem o R² de {r2_comp:.2f}, o MAE de {mae_comp:.2f} e o MSR de {mae_comp:.2f}.')